In [29]:
from torchvision.transforms.transforms import ToTensor
import torch 
import torch.optim as optim
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import time
import math
import json
from sklearn.metrics import accuracy_score

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [20]:
transform = transforms.Compose([
    transforms.ToTensor(),
    nn.Flatten(start_dim=0),
])

train_ds= torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_ds= torchvision.datasets.MNIST(root='./data', train=False, transform=transform)

train_dl = torch.utils.data.DataLoader(dataset=train_ds, batch_size=128, shuffle=True)
test_dl = torch.utils.data.DataLoader(dataset=test_ds, batch_size=128, shuffle=False)

In [25]:
class MultiLayerPerceptron(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(784, 100)
    self.fc2 = nn.Linear(100, 10)

  def forward(self, x):
    x = self.fc1(x)
    x = torch.relu(x)
    x = self.fc2(x)
    return x

# Training

In [38]:
def train_epoch(model, train_dl, epoch):
    model.train()
    train_loss = 0
    tic_train_epoch = time.time()

    for data, target in train_dl:
    
        optimizer.zero_grad()
        output = model(data.to(device))
        loss = criterion(output, target.to(device))
        loss.backward()
        optimizer.step()
    
        train_loss += loss.item()
            
    train_loss = train_loss / len(train_dl)
    toc_train_epoch = time.time()
    time_per_epoch = toc_train_epoch - tic_train_epoch
    print('\nEpoch: {} \tTraining Loss: {:.6f} \tEpoch Time: {:.2f}s'.format(
    epoch+1, 
    train_loss,
    time_per_epoch,
    ))
    metrics = {
        "epoch_train_time": time_per_epoch,
        "epoch_train_loss": train_loss
    }
    return metrics

In [39]:
def eval_epoch(model, test_dl, epoch):
    model.eval()
    tic_eval = time.time()

    test_loss = 0    
    preds = []
    labels = []
    for data, target in test_dl:
        with torch.no_grad():
            output = model(data.to(device))
            loss = criterion(output, target.to(device))

            test_loss += loss.item()

            pred = torch.argmax(output, dim=-1).tolist()
            preds.extend(pred)
            labels.extend(target.tolist())
    toc_eval = time.time()

    acc = accuracy_score(labels, preds)
    print(f'Epoch: {epoch + 1 } \t Test Loss: {test_loss / len(test_dl):.6f} \tEpoch Time: {toc_eval - tic_eval:.2f}s')

    num_eval_batches = len(test_dl)

    average_batch_inference_time = (1000 * (toc_eval - tic_eval) / num_eval_batches) # in ms

    print("Average Batch Inference Time: {:.2f}ms".format(average_batch_inference_time))
    metrics = {
        "eval_accuracy": acc,
        "average_batch_inference_time": average_batch_inference_time
    }
    return metrics

In [40]:
model = MultiLayerPerceptron().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001, betas=[0.9, 0.999])

n_epochs = 10

In [41]:
tic_total_train = time.time()

train_times = []
for epoch in range(10):
    train_metrics = train_epoch(model, train_dl, epoch)
    eval_metrics = eval_epoch(model, test_dl, epoch)
    train_times.append(train_metrics["epoch_train_time"])
toc_total_train = time.time()
total_train_time = toc_total_train - tic_total_train

print("\nTotal Training Time: {:.2f}s".format(total_train_time))
print(f"Average epoch train time: {np.mean(train_times):.2f} s")


Epoch: 1 	Training Loss: 0.437623 	Epoch Time: 10.23s
Epoch: 1 	 Test Loss: 0.238174 	Epoch Time: 1.53s
Average Batch Inference Time: 19.35ms

Epoch: 2 	Training Loss: 0.209079 	Epoch Time: 10.13s
Epoch: 2 	 Test Loss: 0.173932 	Epoch Time: 1.54s
Average Batch Inference Time: 19.55ms

Epoch: 3 	Training Loss: 0.153987 	Epoch Time: 10.28s
Epoch: 3 	 Test Loss: 0.134340 	Epoch Time: 1.55s
Average Batch Inference Time: 19.60ms

Epoch: 4 	Training Loss: 0.120993 	Epoch Time: 10.25s
Epoch: 4 	 Test Loss: 0.116795 	Epoch Time: 1.53s
Average Batch Inference Time: 19.41ms

Epoch: 5 	Training Loss: 0.097512 	Epoch Time: 10.29s
Epoch: 5 	 Test Loss: 0.105389 	Epoch Time: 1.55s
Average Batch Inference Time: 19.58ms

Epoch: 6 	Training Loss: 0.081829 	Epoch Time: 10.28s
Epoch: 6 	 Test Loss: 0.101438 	Epoch Time: 1.55s
Average Batch Inference Time: 19.60ms

Epoch: 7 	Training Loss: 0.068962 	Epoch Time: 10.42s
Epoch: 7 	 Test Loss: 0.087473 	Epoch Time: 1.53s
Average Batch Inference Time: 19.42ms

In [42]:
# export JSON
metrics = {
    "model_name": "MLP",
    "framework_name": "PyTorch",
    "dataset": "MNIST Digits",
    "task": "classification",
    "total_training_time": total_train_time,         # in seconds
    "average_epoch_training_time": np.mean(train_times),  # in seconds
    "average_batch_inference_time": eval_metrics["average_batch_inference_time"],  # in milliseconds
    "final_training_loss": train_metrics["epoch_train_loss"],
    "final_evaluation_accuracy": eval_metrics["eval_accuracy"],
}

with open("m1-pytorch-mlp.json", "w") as outfile:
    json.dump(metrics, outfile)


In [43]:
metrics

{'model_name': 'MLP',
 'framework_name': 'PyTorch',
 'dataset': 'MNIST Digits',
 'task': 'classification',
 'total_training_time': 117.8952624797821,
 'average_epoch_training_time': 10.246275472640992,
 'average_batch_inference_time': 19.221951689901232,
 'final_training_loss': 0.04578290575309031,
 'final_evaluation_accuracy': 0.9781}